In [1]:
import pandas as pd

df = pd.read_csv("final_handled_outlier.csv")
df.dtypes


player_id                int64
season_name             object
competition_name        object
team_name               object
nb_in_group              int64
nb_on_pitch              int64
goals                  float64
assists                  int64
own_goals                int64
subed_in                 int64
subed_out                int64
yellow_cards             int64
second_yellow_cards      int64
direct_red_cards         int64
penalty_goals            int64
minutes_played         float64
goals_conceded           int64
clean_sheets             int64
player_name             object
date_of_birth           object
height                 float64
citizenship             object
is_eu                    int64
position                object
foot                    object
joined                  object
contract_expires        object
date_unix               object
value                  float64
injury_reason           object
days_missed            float64
games_missed           float64
vader_po

In [2]:
df = pd.read_csv("final_handled_outlier.csv")

date_cols = [
    'date_of_birth','joined','contract_expires','date_unix','game_date','tweet_date'
]

for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')


# Integer-like (counts / discrete values)
int_cols = [
    'player_id','nb_in_group','nb_on_pitch','goals','assists','own_goals',
    'subed_in','subed_out','yellow_cards','second_yellow_cards',
    'direct_red_cards','penalty_goals',
    'goals_conceded','clean_sheets','days_missed','games_missed'
]

# Float-like (continuous / sentiment / monetary)
float_cols = [
    'height','tb_polarity','vader_polarity','value','minutes_played'
]

for col in int_cols + float_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

for col in int_cols:
    df[col] = df[col].round().astype('Int64')

for col in float_cols:
    df[col] = df[col].astype('float64')


text_cols = [
    'player_name','citizenship','position',
    'team_name','competition_name',
    'injury_reason','season_name'
]

for col in text_cols:
    df[col] = df[col].astype(str).str.strip()


categorical_cols = ['foot', 'vader_emotion', 'tb_emotion']
for col in categorical_cols:
    df[col] = df[col].astype(str)
for col in categorical_cols:
    df[col] = df[col].str.strip()
for col in categorical_cols:
    df[col] = df[col].str.lower()


# Save the cleaned file
df.to_csv("final_preprocess_data.csv", index=False)

In [3]:
import pandas as pd

# 1️⃣ Load the dataset
df = pd.read_csv("final_preprocess_data.csv")
df.dtypes

player_id                int64
season_name             object
competition_name        object
team_name               object
nb_in_group              int64
nb_on_pitch              int64
goals                    int64
assists                  int64
own_goals                int64
subed_in                 int64
subed_out                int64
yellow_cards             int64
second_yellow_cards      int64
direct_red_cards         int64
penalty_goals            int64
minutes_played         float64
goals_conceded           int64
clean_sheets             int64
player_name             object
date_of_birth           object
height                 float64
citizenship             object
is_eu                    int64
position                object
foot                    object
joined                  object
contract_expires        object
date_unix               object
value                  float64
injury_reason           object
days_missed              int64
games_missed             int64
vader_po

In [4]:
df['is_eu'] = df['is_eu'].astype(int)

# Save the cleaned file
df.to_csv("final_preprocess_data.csv", index=False)

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

df = pd.read_csv("final_preprocess_data.csv")

# 1️⃣ Identify categorical columns
categorical_cols = ["foot", "vader_emotion", "tb_emotion", "position"]

# 2️⃣ Clean raw text values
for col in categorical_cols:
    df[col] = df[col].astype(str).str.strip()   # remove spaces

# 4️⃣ One-hot encode the clean categorical columns
encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
encoded_array = encoder.fit_transform(df[categorical_cols])

# Optional: Create a DataFrame with encoded column names
encoded_df = pd.DataFrame(
    encoded_array,
    columns=encoder.get_feature_names_out(categorical_cols),
    index=df.index
)

# (Optional) Merge encoded columns back into original df
df = pd.concat([df.drop(columns=categorical_cols), encoded_df], axis=1)

# Save the cleaned file
df.to_csv("final_preprocess_data.csv", index=False)


In [9]:
import pandas as pd

# Load the file
df = pd.read_csv("final_preprocess_data.csv")
# -----------------------------
# High-cardinality categorical columns
# -----------------------------
high_card_cat_cols = ['competition_name', 'team_name', 'citizenship']

for col in high_card_cat_cols:
    
    # Convert to string & clean
    df[col] = df[col].astype(str).str.strip()
    df[col] = df[col].replace(
        ['nan', 'None', 'none', 'null', 'Null', ''], 
        'missing'
    )

    # Frequency encode
    freq_map = df[col].value_counts()
    df[col + "_freq"] = df[col].map(freq_map)

    # Drop original column
    df.drop(columns=[col], inplace=True)

 # Save the cleaned file
df.to_csv("final_preprocess_data.csv", index=False)


In [6]:
# Load dataset
df = pd.read_csv("final_preprocess_data.csv")

#2. Convert date columns to datetime

date_cols = ["date_of_birth", "joined", "contract_expires", "game_date", "tweet_date", "date_unix"]
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')


# Player age at market value date
df['age'] = (df['date_unix'] - df['date_of_birth']).dt.days // 365

# Contract remaining days
df['contract_remaining_days'] = (df['contract_expires'] - df['date_unix']).dt.days
df['contract_remaining_days'] = df['contract_remaining_days'].clip(lower=0)

# Days since joined club
df['days_since_joined'] = (df['date_unix'] - df['joined']).dt.days
df['days_since_joined'] = df['days_since_joined'].clip(lower=0)

# Days since last game
df['days_since_game'] = (df['date_unix'] - df['game_date']).dt.days
df['days_since_game'] = df['days_since_game'].clip(lower=0)

# Days since last tweet
df['days_since_tweet'] = (df['date_unix'] - df['tweet_date']).dt.days
df['days_since_tweet'] = df['days_since_tweet'].clip(lower=0)

# Save the cleaned file
df.to_csv("final_preprocess_data.csv", index=False)


In [7]:
import pandas as pd

# Load the dataset
df = pd.read_csv("final_preprocess_data.csv")

# ---- Convert to datetime ----
df['joined'] = pd.to_datetime(df['joined'], errors='coerce')
df['contract_expires'] = pd.to_datetime(df['contract_expires'], errors='coerce')
df['date_unix'] = pd.to_datetime(df['date_unix'], errors='coerce')

# ---- JOINED DATE FEATURES ----
df['joined_year'] = df['joined'].dt.year
df['joined_month'] = df['joined'].dt.month

# ---- CONTRACT EXPIRES FEATURES ----
df['contract_expires_year'] = df['contract_expires'].dt.year
df['contract_expires_month'] = df['contract_expires'].dt.month

# Time left on contract
df['contract_years_left'] = df['contract_remaining_days'] // 365
df['contract_years_left'] = df['contract_years_left'].clip(lower=0)


# Save the cleaned dataset
df.to_csv("final_preprocess_data.csv", index=False)


In [8]:

# Load the dataset
df = pd.read_csv("final_preprocess_data.csv")

# Convert both to datetime
df['game_date'] = pd.to_datetime(df['game_date'], errors='coerce')
df['tweet_date'] = pd.to_datetime(df['tweet_date'], errors='coerce')

# ---- GAME DATE FEATURES ----
df['game_year'] = df['game_date'].dt.year
df['game_month'] = df['game_date'].dt.month

# ---- TWEET DATE FEATURES ----
df['tweet_year'] = df['tweet_date'].dt.year
df['tweet_month'] = df['tweet_date'].dt.month

# Save the cleaned file
df.to_csv("final_preprocess_data.csv", index=False)

In [10]:
# Load the dataset
df = pd.read_csv("final_preprocess_data.csv")

# List of date columns
date_cols = ["date_of_birth", "joined", "contract_expires", "game_date", "tweet_date", "date_unix"]

# Drop these columns
df = df.drop(columns=date_cols)

# Save the cleaned dataset
df.to_csv("final_preprocess_data.csv", index=False)

In [11]:
import pandas as pd

# 1️⃣ Load the dataset
df = pd.read_csv("final_preprocess_data.csv")

# 2️⃣ Extract first two digits of season start (e.g., 18/19 → 18)
df['season'] = df['season_name'].str[:2].astype(int)

# 3️⃣ Fix century ambiguity
# 00–25 → 2000+
# 26–99 → 1900+
df['season'] = df['season'].apply(
    lambda x: 2000 + x if x <= 25 else 1900 + x
)

# 4️⃣ Drop original season column
df = df.drop(columns=['season_name'])

# 5️⃣ Save the updated dataset
df.to_csv("final_preprocess_data.csv", index=False)



In [12]:
import pandas as pd

# 1️⃣ Load the dataset
df = pd.read_csv("final_preprocess_data.csv")

# Ensure column is string and normalized
df['injury_reason'] = df['injury_reason'].astype(str).str.strip().str.lower()

# Replace values
df['injury_reason'] = df['injury_reason'].apply(
    lambda x: 'no injury' if x == 'no injury' else 'injured'
)

# 5️⃣ Save the updated dataset
df.to_csv("final_preprocess_data.csv", index=False)

In [13]:
df.rename(columns={'injury_reason': 'injury'}, inplace=True)

# 5️⃣ Save the updated dataset
df.to_csv("final_preprocess_data.csv", index=False)

In [14]:
df['injury'] = df['injury'].map({'no injury': 0, 'injured': 1})

# 5️⃣ Save the updated dataset
df.to_csv("final_preprocess_data.csv", index=False)

In [15]:
import pandas as pd

# 1️⃣ Load the dataset
df = pd.read_csv("final_preprocess_data.csv")
df.dtypes

player_id                int64
nb_in_group              int64
nb_on_pitch              int64
goals                    int64
assists                  int64
                         ...  
tweet_month              int64
competition_name_freq    int64
team_name_freq           int64
citizenship_freq         int64
season                   int64
Length: 68, dtype: object

In [16]:
import pandas as pd

# Load your dataset
df = pd.read_csv("final_preprocess_data.csv")

# Create a table with column names and data types
dtype_table = pd.DataFrame({
    'Column Name': df.columns,
    'Data Type': df.dtypes.values
})

# Save the table to CSV
dtype_table.to_csv("column_dtypes.csv", index=False)

print("Column data types saved to 'column_dtypes.csv'")



Column data types saved to 'column_dtypes.csv'


In [17]:
import pandas as pd

# Load dataset
df = pd.read_csv("final_preprocess_data.csv")

# -------------------------
# Columns to convert to int
# -------------------------
int_cols = [
    'player_id', 'nb_in_group', 'nb_on_pitch', 'goals', 'assists', 'own_goals',
    'subed_in', 'subed_out', 'yellow_cards', 'second_yellow_cards', 'direct_red_cards',
    'penalty_goals', 'goals_conceded', 'clean_sheets',
    'days_missed', 'games_missed', 'age', 'contract_remaining_days',
    'days_since_joined', 'days_since_game', 'days_since_tweet',
    'joined_year', 'joined_month', 'tenure_years',
    'contract_year', 'contract_month', 'contract_years_left',
    'game_year', 'game_month', 'tweet_year', 'tweet_month', 'season_start',
    # Add the binary / OHE columns
    'foot_both', 'foot_left', 'foot_right',
    'vader_emotion_negative','vader_emotion_neutral','vader_emotion_positive',
    'tb_emotion_negative','tb_emotion_neutral','tb_emotion_positive',
    'position_attack','position_attack - centre-forward','position_attack - left winger',
    'position_attack - right winger','position_attack - second striker',
    'position_defender','position_defender - centre-back','position_defender - left-back',
    'position_defender - right-back','position_defender - sweeper','position_goalkeeper',
    'position_midfield','position_midfield - attacking midfield','position_midfield - central midfield',
    'position_midfield - defensive midfield','position_midfield - left midfield','position_midfield - right midfield',
    'injury', 'competition_name_freq', 'team_name_freq', 'citizenship_freq'
]

# -------------------------
# Columns to convert to float
# -------------------------
float_cols = [
    'height', 'value', 'vader_polarity', 'tb_polarity', 'minutes_played'
]

# -------------------------
# Convert int columns safely
# -------------------------
for col in int_cols:
    if col in df.columns:
        df[col] = df[col].fillna(0).astype(int)

# -------------------------
# Convert float columns safely
# -------------------------
for col in float_cols:
    if col in df.columns:
        df[col] = df[col].astype(float)

# -------------------------
# Check data types after conversion
# -------------------------
print(df.dtypes)

# -------------------------
# Save updated dataset
# -------------------------
df.to_csv("final_preprocess_data.csv", index=False)


player_id                int64
nb_in_group              int64
nb_on_pitch              int64
goals                    int64
assists                  int64
                         ...  
tweet_month              int64
competition_name_freq    int64
team_name_freq           int64
citizenship_freq         int64
season                   int64
Length: 68, dtype: object


In [18]:
import pandas as pd

# Load your dataset
df = pd.read_csv("final_preprocess_data.csv")

# Create a table with column names and data types
dtype_table = pd.DataFrame({
    'Column Name': df.columns,
    'Data Type': df.dtypes.values
})

# Save the table to CSV
dtype_table.to_csv("column_dtypes_final.csv", index=False)

print("Column data types saved to 'column_dtypes.csv'")



Column data types saved to 'column_dtypes.csv'
